### Target

**Find the articles which are judged as Xinhua net but not is Xinhua net. **

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import jieba

In [5]:
def cut(string): return ' '.join(jieba.cut(string))

In [6]:
content = pd.read_csv('dataset/sqlResult_1558435.csv', encoding='gb18030')

In [7]:
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [8]:
content['content'] = content['content'].fillna('')
content['content'] = content['content'].apply(cut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/m2/x2_2zr4j2sg4v43q2dr50ppr0000gn/T/jieba.cache
Loading model cost 0.687 seconds.
Prefix dict has been built succesfully.


In [9]:
xinhuashe_news = content[content['source'].str.contains('新华', na=False)]

In [10]:
len(xinhuashe_news)

78855

In [11]:
len(content)

89611

## Benchmark

In [12]:
len(xinhuashe_news) / len(content)

0.8799700929573379

## TF-IDF Vectorized

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
content['is_xinhua'] = np.where(content['source'].str.contains('新华'), 1, 0)

In [86]:
content['is_xinhua']

1

In [16]:
X_content = content['content'].values

In [17]:
len(X_content)

89611

In [18]:
y = content['is_xinhua'].values

In [19]:
from sklearn.model_selection import train_test_split

## Get X_title tfidf vector

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vectorizer = TfidfVectorizer(max_features=30000)

In [22]:
X_content = vectorizer.fit_transform(X_content)

In [23]:
X_content.shape

(89611, 30000)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_content, y, test_size=0.25, random_state=0)

In [25]:
#shuffle the order
indices = np.arange(len(y))
np.random.shuffle(indices)
test_split = 0.85
valid_split = 0.85

In [26]:
validation_indices = indices[int(len(indices) * valid_split):]

In [27]:
len(validation_indices)

13442

In [28]:
test_train_indices = indices[:int(len(indices) * valid_split)]

In [29]:
len(test_train_indices)

76169

In [30]:
train_indices = test_train_indices[: int(len(test_train_indices)*test_split)]

In [31]:
len(train_indices)

64743

In [32]:
test_indices = test_train_indices[int(len(test_train_indices)*test_split):]

In [33]:
len(test_indices)

11426

In [34]:
X_train, X_test, X_val = (
    X_content[train_indices], X_content[test_indices], 
    X_content[validation_indices])

In [35]:
y_train, y_test, y_val = (
    y[train_indices], y[test_indices], y[validation_indices])

In [36]:
from sklearn.naive_bayes import GaussianNB

In [37]:
clf = GaussianNB()

In [38]:
from sklearn import datasets

In [39]:
clf.fit(X_train.toarray(), y_train)

GaussianNB(priors=None)

In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
def get_performance(clf, X_, y_):
    #random_indices = np.random.choice(np.arange(len(y_)), sample)
    #sub_x = X_[random_indices]
    #sub_y = y_[random_indices]
    y_hat = clf.predict(X_.toarray())
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

In [42]:
from sklearn.metrics import precision_score

from sklearn.metrics import recall_score

from sklearn.metrics import roc_auc_score


In [43]:
get_performance(clf, X_=X_train, y_=y_train)

percision is: 0.9998162710369662
recall is: 0.955456061803178
roc_auc is: 0.9770860175477113
confusion matrix: 
[[ 7778    10]
 [ 2537 54418]]


In [44]:
get_performance(clf, X_=X_test, y_=y_test)

percision is: 0.9750582986920815
recall is: 0.9566298617328161
roc_auc is: 0.8887300801744926
confusion matrix: 
[[1127  246]
 [ 436 9617]]


In [45]:
test_predict = clf.predict(X_test.toarray())

In [46]:
precision_score(y_test, test_predict)

0.9750582986920815

In [47]:
recall_score(y_test, test_predict)

0.9566298617328161

In [48]:
roc_auc_score(y_test, test_predict)

0.8887300801744926


### Find the most similar article

In [49]:
result = []
for i in range(len(test_predict)):
    if test_predict[i]==1 and y_test[i] !=1:
        result.append(i)        

In [81]:
y_test[11382]

0

In [51]:
#抄袭判定（利用余弦距离计算相似性）
def CosineDistance(x,y):
    result = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return result

In [52]:
from sklearn.feature_extraction.text import CountVectorizer 

In [53]:
test_data_list = []
for i in result:
    test_data_list.append( "".join((cut(str(content.iloc[i].content)))))

In [54]:
y_test2 = y[test_indices]

In [55]:
result2 = []
for i, j in enumerate(y[test_indices]):
    if j == 1:
        result2.append(test_indices[i])
        

In [56]:
all_data_list = []
for i in result2:
    all_data_list.append("".join((cut(str(content.iloc[i].content)))))

In [57]:
count_v1 = CountVectorizer()#将预测错误的文本中的词语转换为词频矩阵

In [58]:
sample = count_v1.fit_transform(test_data_list)

In [59]:
sample = sample.toarray()

In [60]:
sample.shape

(246, 14770)

In [61]:
count_v2 = CountVectorizer(vocabulary=count_v1.vocabulary_)

In [62]:
all = count_v2.fit_transform(all_data_list)

In [63]:
all.shape

(10053, 14770)

In [64]:
all = all.toarray()

In [65]:
Distance = []
for i in range(len(result)):
    distance_for_one = []
    for k in range(len(result2)):
        distance_for_one.append(CosineDistance(sample[i][:],all[k][:]))
    Distance.append([max(distance_for_one), result[i], result2[distance_for_one.index(max(distance_for_one))]])  

/Users/wangzifeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [127]:
[element for element in Distance if element[0]>=0.6 and element[0]<=0.8]

[[0.6282387374277474, 881, 49723],
 [0.683538526126733, 3426, 49723],
 [0.6165630281176733, 6566, 11860],
 [0.6177304258908716, 7733, 83543],
 [0.6408716301164642, 10970, 19893]]

In [128]:
content.iloc[3426]

id                                                       86191
author                                                     NaN
source                                                   参考消息网
content      参考消息 网 6 月 21 日 报道   西班牙 《 发展 报 》 网站 6 月 20 日 ...
feature      {"type":"国际观点","site":"参考消息","commentNum":"0",...
title                                          西媒：六大关键人物领衔脱欧谈判
url          http://column.cankaoxiaoxi.com/2017/0621/21359...
is_xinhua                                                    0
Name: 3426, dtype: object

In [129]:
content.iloc[49723]

id                                                       39889
author                                                      桂涛
source                                                     新华社
content      　 　 新华社 伦敦 ５ 月 ３ 日电 （ 记者 桂涛 ） 英国首相 特雷莎 · 梅 ３ 日...
feature      {"type":"政治法律","site":"新华社","url":"http://home...
title                                     （国际）英国首相批评欧盟企图影响６月大选
url          http://home.xinhua-news.com/gdsdetailxhsnew/20...
is_xinhua                                                    1
Name: 49723, dtype: object